In [1]:
# --- TEST DATA_LOADER ---

from src import data_loader as dl
import pandas as pd

filepath = r'data/U13271915_20250101_20251029.csv'
output_excel_path = r'data/report_output.xlsx'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 150)

print(f"Loading report from: {filepath}...")
results = dl.load_ibkr_report(filepath)

if results:
    print("\nReport loaded successfully.")
    
    try:
        # Use pd.ExcelWriter to save both DataFrames to one file
        # on separate sheets.
        with pd.ExcelWriter(output_excel_path) as writer:
            results['initial_state'].to_excel(writer, sheet_name='initial_state', index=False)
            results['events'].to_excel(writer, sheet_name='events', index=False)
            results['financial_info'].to_excel(writer, sheet_name='financial_info', index=False)
            
        print(f"Successfully saved data to: {output_excel_path}")
        
    except ImportError:
        print("\n--- ERROR ---")
        print("To save to Excel, you need the 'openpyxl' library.")
        print("Please install it by running this in your terminal:")
        print("pip install openpyxl")
    except Exception as e:
        print(f"\nAn error occurred while saving to Excel: {e}")

    # --- 4. Print results to console as before ---
    print("\n" + "="*80 + "\n")
    print("--- 1. Initial State DataFrame ---")
    print(results['initial_state'])
    
    print("\n" + "="*80 + "\n")
    print("--- 2. Master Event Log ---")
    print(results['events'])

else:
    print("Data loading failed. No results to save.")

Loading report from: data/U13271915_20250101_20251029.csv...

Report loaded successfully.
Successfully saved data to: data/report_output.xlsx


--- 1. Initial State DataFrame ---
   symbol asset_category currency      quantity  value_native
0    ANFO          Stock      CHF      0.000000      0.000000
1    ASML          Stock      USD      2.000000   1386.160000
2   CHDVD          Stock      CHF     45.000000   7106.400000
3     DGE          Stock      GBP     56.000000   1421.000000
4    EMBC          Stock      CHF      0.000000      0.000000
5    FLIN          Stock      USD    192.000000   7276.800000
6    FONC          Stock      CHF      0.000000      0.000000
7      HO          Stock      EUR      0.000000      0.000000
8    IBKR          Stock      USD      3.036800    536.511456
9    IHYU          Stock      CHF      0.000000      0.000000
10   IUSC          Stock      CHF     86.000000   7833.740000
11    IVV          Stock      USD     12.000000   7064.160000
12    KGX      

In [2]:
import pandas as pd
import os
# Assuming your modules are accessible in the path
from src import data_loader as dl
from src import data_processor as dp
from src import market_data_loader as mdl

# --- 1. Setup Paths ---
input_csv_path = r'data/U13271915_20250101_20251029.csv'
output_excel_path = r'data/market_data.xlsx'

# Check if input file exists before proceeding
if not os.path.exists(input_csv_path):
    print(f"Error: Input file not found at {input_csv_path}")
else:
    print(f"Input file found: {input_csv_path}")

    # --- 2. Run the Pipeline ---

    print("\n1. Loading IBKR Report...")
    # Load raw data
    data_package = dl.load_ibkr_report(input_csv_path)

    print("2. Applying Split Adjustments...")
    # Process splits/events
    processed_data = dp.apply_split_adjustments(data_package)

    print("3. Downloading Market Data...")
    # Download from Yahoo Finance
    # This returns 'market_data_map' (Dictionary of DataFrames)
    final_package = mdl.load_market_data(processed_data)

    # --- 3. Inspect and Save Results ---

    if not final_package:
        print("Warning: No market data was downloaded. Check internet connection or ticker logic.")
    else:
        print(f"\nSuccess! Downloaded data for {len(final_package)} assets.")
        
        # A. Quick Inspection of first asset
        first_symbol = list(final_package.keys())[0]
        print(f"--- Sample Data for: {first_symbol} ---")
        print(final_package[first_symbol].head())
        print("-" * 30)

        # B. Export to Excel
        print(f"4. Saving to {output_excel_path}...")
        
        # We use ExcelWriter to save multiple dataframes to one file (one sheet per symbol)
        try:
            with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
                for symbol, df_data in final_package.items():
                    # Excel sheet names are limited to 31 chars and cannot contain [] : * ? / \
                    clean_sheet_name = str(symbol).replace('/', '_')[:31]
                    df_data.to_excel(writer, sheet_name=clean_sheet_name)
            print("Excel file saved successfully.")
            
        except Exception as e:
            print(f"Error saving Excel file: {e}")

Input file found: data/U13271915_20250101_20251029.csv

1. Loading IBKR Report...
2. Applying Split Adjustments...
3. Downloading Market Data...

Success! Downloaded data for 26 assets.
--- Sample Data for: ANFO ---
Price            Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2020-01-03  74.250000  77.099998  74.250000  76.800003  70.568115  21264.0
2020-01-06  76.650002  78.500000  76.099998  78.500000  72.130157  34554.0
2020-01-07  78.599998  79.300003  78.599998  79.300003  72.865250  72834.0
2020-01-08  78.599998  80.000000  78.550003  80.000000  73.508446  27223.0
2020-01-09  80.000000  80.000000  79.099998  79.650002  73.186852  21843.0
------------------------------
4. Saving to data/market_data.xlsx...
Excel file saved successfully.
